In [1]:
import os
from multiprocessing import Pool

import numpy as np

from src.vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from src.vrp_study.pdptw_model.routing_model import find_optimal_paths
from src.vrp_study.data_loader import parse_data
from src.vrp_study.configs import ModelConfig
from vrp_study.data_model import Tariff, Cargo, Node
from vrp_study.routing_manager import RoutingManager
from src.vrp_study.pdptw_model.solution_builder import SolutionBuilder
import pickle


In [2]:
def func(du, dv):
    return np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2)


def calc(data):
    u, du, p2coordinate = data
    return {(u, v): func(du, dv) for v, dv in p2coordinate.items()}


def rm(
        depo: Node,
        cargos: list[Cargo],
        tariff: Tariff,
        p: Pool
) -> RoutingManager:
    p2coordinates = {}

    p2coordinates.update({
        crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
    })
    p2coordinates[depo.id] = depo.coordinates

    res = list(p.imap_unordered(calc, [(u, du, p2coordinates) for u, du in p2coordinates.items()]))
    distance_matrix = {}
    time_matrix = {}
    for r in res:
        for k, v in r.items():
            distance_matrix[k] = v
            time_matrix[k] = v

    routing_manager = PDRoutingManagerBuilder(
        distance_matrix=distance_matrix,
        time_matrix=time_matrix,
        model_config=ModelConfig(max_execution_time_minutes=1)
    )

    routing_manager.add_cargos(cargos)
    routing_manager.add_tariff(tariff)

    routing_manager.add_depo(depo)

    routing_manager = routing_manager.build()
    return routing_manager

In [3]:
def rms_calc(data):
    routing_manager, benchmark_type, name = data
    sol = find_optimal_paths(routing_manager, SolutionBuilder(
        save_graph=True, name = f'{benchmark_type}_{name}'
    ))

    sols = []
    for s in sol[0]:
        if len(s) > 0:
            sols.append([routing_manager.nodes()[i].routing_node.id for i in s[1:-1]])
    with open(f'../data/results/{benchmark_type}_{name}.pkl', 'wb') as f:
        pickle.dump(sols, f)
    del routing_manager, benchmark_type, name, sol, sols


In [4]:
from tqdm import tqdm
from loguru import logger as log
log.remove()
NUM_WORKERS = 4
MAX_SIZE = 10 # создаем столько менеджеров. потом в парралель NUM_WORKERS  решают

with Pool(NUM_WORKERS ) as p:
    for benchmark_type in ['pdp_100']:#os.listdir('../data/Li & Lim benchmark'):
        rms = []
        for name in tqdm(os.listdir(f'../data/Li & Lim benchmark/{benchmark_type}')):
            depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/{benchmark_type}/{name}')
            rms.append((rm(depo, cargos, tariff, p), benchmark_type, name))
            if len(rms) == MAX_SIZE:
                rr = list(p.imap_unordered(rms_calc, rms))
                rms = []
        break
    if len(rms) > 0:
        rr = list(p.imap_unordered(rms_calc, rms))
        rms = []

100%|██████████| 56/56 [02:38<00:00,  2.82s/it]
